In [1]:
# http://compphysics.github.io/ComputationalPhysics/doc/pub/vmc/html/vmc.html

from sympy import symbols, diff, exp, sqrt
x, y, z, Z = symbols('x y z Z')
r = sqrt(x*x + y*y + z*z)
r
phi = (Z*r - 2)*exp(-Z*r/2)
phi


(Z*sqrt(x**2 + y**2 + z**2) - 2)*exp(-Z*sqrt(x**2 + y**2 + z**2)/2)

In [2]:
diff(phi, x)

-Z*x*(Z*sqrt(x**2 + y**2 + z**2) - 2)*exp(-Z*sqrt(x**2 + y**2 + z**2)/2)/(2*sqrt(x**2 + y**2 + z**2)) + Z*x*exp(-Z*sqrt(x**2 + y**2 + z**2)/2)/sqrt(x**2 + y**2 + z**2)

In [3]:
from sympy import symbols, diff, exp, sqrt, factor, Symbol, printing
x, y, z, Z = symbols('x y z Z')
r = sqrt(x*x + y*y + z*z)
phi = (Z*r - 2)*exp(-Z*r/2)
R = Symbol('r') #Creates a symbolic equivalent of r
#print latex and c++ code
diff(phi, x).factor().subs(r, R)

-Z*x*(Z*r - 4)*exp(-Z*r/2)/(2*r)

In [4]:
from sympy import symbols, diff, exp, sqrt, factor, Symbol, printing
x, y, z, Z = symbols('x y z Z')
r = sqrt(x*x + y*y + z*z)
phi = (Z*r - 2)*exp(-Z*r/2)
R = Symbol('r') #Creates a symbolic equivalent of r
(diff(diff(phi, x), x) + diff(diff(phi, y), y) + diff(diff(phi, z), z)).factor().subs(r, R)
# Collect the Z values
(diff(diff(phi, x), x) + diff(diff(phi, y), y) +diff(diff(phi, z), z)).factor().collect(Z).subs(r, R)
# Factorize also the r**2 terms
(diff(diff(phi, x), x) + diff(diff(phi, y), y) + diff(diff(phi, z), z)).factor().collect(Z).subs(r, R).subs(r**2, R**2).factor()

Z*(Z*r - 8)*(Z*r - 2)*exp(-Z*r/2)/(4*r)

In [5]:
nDimensions = 3
charge = 2
stepLength = 1.0
nParticles = 2
# step length and square inverse
h = 0.001
h2 = 1000000
alpha = 1.75
nCycles = 1000000



In [6]:
import numpy as np
from tqdm import tqdm



def wave_func(r): 
    # exp(-alpha(r1+r2))
    argument = 0.
    for i in range(nParticles):
        r_single_particle = 0.
        for j in range(nDimensions):
            r_single_particle += r[i, j]**2
        argument += np.sqrt(r_single_particle)
    
    return np.exp(-argument * alpha)

def local_energy(r):
    wave_func_current = wave_func(r)
    r_plus = np.copy(r)
    r_minus = np.copy(r)
    
    # Kinetic Energy (Central diff.?)
    T = 0.
    for i in range(nParticles):
        for j in range(nDimensions):
            r_plus[i,j]  += h
            r_minus[i,j] += h
            wave_func_plus  = wave_func(r_plus)
            wave_func_minus = wave_func(r_minus)
            T -= (wave_func_minus + wave_func_plus - 2 * wave_func_current)
            r_plus[i,j] = r[i,j]
            r_minus[i,j] = r[i,j]
    
    T *= 0.5 * h2 / wave_func_current
    
    # Potential Energy
    U = 0.
    for i in range(nParticles):
        r_single_particle = 0.
        for j in range(nDimensions):
            r_single_particle += r[i,j]**2
        U -= charge / np.sqrt(r_single_particle)

    # Potentaial Energy due to r12
    V = 0.
    for i in range(nParticles):
        for j in range(i+1, nParticles):
            r12 = 0.
            for k in range(nDimensions):
                r12 += (r[i,k] - r[j,k])**2
            V += 1 / np.sqrt(r12)
    
    return T + U + V


# def runMonteCarloIntegration():
r_old = np.zeros((nParticles, nDimensions))
r_new = np.zeros((nParticles, nDimensions))
energy = 0.
all_energy = []

for i in range(nParticles):
    for j in range(nDimensions):
        r_old[i,j] = stepLength * (np.random.rand() - 0.5)

r_new = np.copy(r_old)

accept = 0

for cycle in tqdm(range(nCycles)):

    wave_func_old = wave_func(r_old)

    for i in range(nParticles):
        for j in range(nDimensions):
            r_new[i,j] = r_old[i,j] + stepLength * (np.random.rand() - 0.5)
        
        wave_func_new = wave_func(r_new)

        # Metropolis algorithm
        ran = np.random.rand()
        if ran <= (wave_func_new**2)/(wave_func_old**2):
            for i in range(nParticles):
                for j in range(nDimensions):
                    r_old[i,j] = r_new[i,j]
            wave_func_old = wave_func_new
            accept += 1
        else:
            for j in range(nDimensions):
                r_new[i,j] = r_old[i,j]
    
        # Update energies(?)
        deltaE = local_energy(r_new)/(nCycles * nParticles)
        energy += deltaE
        all_energy.append(energy)

100%|██████████| 1000000/1000000 [01:20<00:00, 12426.63it/s]


In [7]:
energy*2*13.6

-17201.560248519403

In [8]:
energy

-632.4103032543899